<h2 align="center"> Data Mining and Machine Learning </h2>
<h3 align="center"> Final Project </h3>
<h2 align="center"> <b> <i> CrashSpot </i> </b> </h2>
<h4 align="center"> Lorenzo Ceccanti matr. 564490 </h4>

### <b> Geografical Clustering with OPTICS</b>

In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.cluster import OPTICS

R_EARTH_KM = 6371.0

df = pd.read_csv(os.path.join('../editedDataset', 'CLEANED_brasilEnglishAggr.csv'))

In [3]:
df.loc[:,"state"].value_counts()

state
MG    61160
SC    55343
PR    52824
RS    32307
RJ    32136
SP    30693
BA    24295
GO    22709
PE    18631
ES    17297
MT    16234
MS    10816
CE    10704
PB    10301
RO    10168
RN     9129
PI     8504
MA     8040
PA     6776
DF     6612
AL     4415
TO     3948
SE     3862
AC     1352
RR     1264
AP      788
AM      589
Name: count, dtype: int64

In [ ]:
import plotly.express as px
import pandas as pd

def plot_map2(data: pd.DataFrame, text: str, clusters_stats):

    cont_scale = [
            [0.0,  "green"],
            [0.1,  "green"],
            [0.2,  "green"],
            [0.3,  "yellow"],
            [0.4,  "yellow"],
            [0.5,  "yellow"],
            [0.6,  "red"],
            [0.7,  "red"],
            [0.8,  "red"],
            [0.9,  "red"],
            [1.0,  "red"]
        ]
    
    fig = px.scatter_mapbox(
        data,
        lat="latitude",
        lon="longitude",
        color = data["labels"].map(clusters_stats.set_index("labels")["hotspot_score"]),
        color_continuous_scale=cont_scale
    )
    fig.update_layout(
        mapbox_style="carto-positron",
        title=f"Map plot ({text})"
    )
    fig.show()

In [ ]:
import plotly.express as px
import pandas as pd

def plot_map(data: pd.DataFrame, text: str, color_col: str, zoom_factor=3):
    if color_col == "labels":
        # Variabile numerica/continua
        cont_scale = [
            [0.0,  "gray"],
            [0.1,  "cyan"],
            [0.2,  "lightblue"],
            [0.3,  "blue"],
            [0.4,  "green"],
            [0.5,  "lightgreen"],
            [0.6,  "yellow"],
            [0.7,  "gold"],
            [0.8,  "orange"],
            [0.9,  "orangered"],
            [1.0,  "red"]
        ]
        fig = px.scatter_map(
            data,
            lat="latitude",
            lon="longitude",
            color=color_col,  # nome colonna
            color_continuous_scale=cont_scale,
            hover_data=["inverse_data", color_col]
        )
    else:
        # Variabile categorica (testuale)
        discrete_map = {
            "Without victims": "green",
            "With injured victims": "orange",
            "With dead victims": "red"
        }
        fig = px.scatter_map(
            data,
            lat="latitude",
            lon="longitude",
            color=color_col,     # nome colonna
            color_discrete_map=discrete_map,
            category_orders={color_col: ["Without victims", "With injured victims", "With dead victims"]},
            hover_data=["inverse_data", color_col]
        )

    fig.update_layout(
        mapbox_style="carto-positron",
        title=f"Map plot ({text})"
    )
    fig.show()


In [6]:
from sklearn import metrics
from sklearn.metrics import davies_bouldin_score
def get_GeoOPTICS_metrics(max_radius, minPts, xi, gpsCoords):

    coords = gpsCoords.copy()
    # Converting in radians
    coords[["latitude","longitude"]] = np.radians(coords[["latitude","longitude"]])

    # If we use OPTICS, we cannot maintain multiple accidents characterized by the *exact* couple (lat, lon)! In that case, 
    # we would obtain 0 in the computation of the reachability distance. 
    # This is one of the known issue of the OPTICS algorithm. For this reason, to fit OPTICS I'll pass a dataframe without duplicates.
    
    # Couting for each duplicate 
    aggr = (coords
       .groupby(["latitude","longitude"], as_index=False)
       .size().rename(columns={"size":"count"}))

    np_coords = aggr[["latitude","longitude"]].to_numpy()
    # Fit
    # min_samples defines the minimum density: how many neighbors are required to consider a point as a core point
    # max_eps: it's not useful to insert a value too much higher than the knee, so we put the knee value (eps_km)
    optics = OPTICS(min_samples=minPts, max_eps=max_radius/R_EARTH_KM, metric='haversine', xi=xi)
    optics.fit(np_coords)
    
    aggr["labels"] = optics.labels_
    
    # Re-assigning the labels to original data
    gps_labeled = coords.merge(
        aggr[["latitude","longitude","labels","count"]],
        on=["latitude","longitude"],
        how="left"
    )
    
    # Preparing a mask to exclude the indexes belonging to outliers
    mask = gps_labeled["labels"].ne(-1) # These are our total labels for OPTICS!

    data_filtered = gps_labeled.loc[mask, ["latitude","longitude"]]
    labels_filtered = gps_labeled.loc[mask, "labels"]
    
    # First metric: Percentage of core points and number of outliers
    count_core_points = np.sum(mask)
    total_points = np.size(mask)
    count_outliers = total_points - count_core_points

    # Second metric: Number of clusters
    labels_all = gps_labeled["labels"].to_numpy()
    number_of_clusters = len(set(labels_all[labels_all >= 0]))
    if number_of_clusters <= 1 or len(np.unique(labels_filtered)) <= 1:
        return "No clusters", ""

    # Third metric: Davies-Bouldin Index
    davies_bouldin_index = davies_bouldin_score(data_filtered, labels_filtered)

    # Fourth metric: Silouette Coefficent
    sil_coefficent = metrics.silhouette_score(data_filtered, labels_filtered, metric='haversine')

    # Fifth metrics: Calinski-Harabasz index
    calinski_index = metrics.calinski_harabasz_score(data_filtered, labels_filtered)

    # I return a dictionary containing the metrics associated with the Eps and MinPts chosen
    df_results = {
        'min_samples': minPts,
        'max_radius': max_radius,
        'xi': xi, 
        'count_core_points': count_core_points,
        'count_outliers': count_outliers,
        'corePoints_outliers_ratio': count_core_points/count_outliers,
        'number_of_clusters': number_of_clusters,
        'davies_bouldin_index': davies_bouldin_index,
        'sil_coefficent': sil_coefficent,
        'calinski_index': calinski_index
    }
    return df_results, labels_all    

In [7]:
def optics_tuning(maxRadiusArr, minPtsArr, xiArr, gpsCoords):
    # It's an array of dictionaries
    arr_tuning_results = []

    for max_eps in maxRadiusArr:
        for minPts in minPtsArr:
            for xi in xiArr:
                df_metric, _ = get_GeoOPTICS_metrics(max_eps, minPts, xi, gpsCoords)
                if isinstance(df_metric, str) == False:
                    arr_tuning_results.append(df_metric)

    df_tuning_results = pd.DataFrame(arr_tuning_results)
    return df_tuning_results

In [8]:
def get_optics_tuning_df(cause, maxRadiusArr, minPtsArr, xiArr, data):
    df_filtered = data.query(f"general_cause_of_accident == '{cause}'").copy()

    gpsCoords = df_filtered[['latitude', 'longitude']].copy()
    tuning_results = optics_tuning(maxRadiusArr, minPtsArr, xiArr, gpsCoords)
    if tuning_results.empty:
        return "No clusters"
    else:
        tuning_results = tuning_results.sort_values(
                by=["corePoints_outliers_ratio", "sil_coefficent", "davies_bouldin_index","calinski_index"],
                ascending=[True, False, True, False]
        )
        return tuning_results.query("corePoints_outliers_ratio > 1.4")

In [9]:
from sklearn.preprocessing import MinMaxScaler
def launch(param, query, cause_to_analyze, maxRadius_to_test, minPtsArr, xi_to_test, max_radiusUser, min_ptsUser, xiUser):
    if param == "City":
        # We select a specific city
        df_selection = df.copy().query(f"city == '{query}'")
    if param == "State":
        # We select a specific state
        df_selection = df.copy().query(f"state == '{query}'")
    
    cause_mapping = {
    "Brake slam": ["Abrupt use of the car's brake"],
    "Minor traffic offense": ["Absence of sinalization",
                              "Disobedience to laws of transit by the pedestrian",
                              "car's on sidewalk"],
    "Traffic offense": ["Driver broke the laws of transit", "Irregular access",
                        "Lane change maneuver",
                        "Stopping at a prohibited place",
                        "The driver passed the next car improperly",
                        "Traffic with a motorcycle (or similar) between lanes",
                        "Acessing the road without seeing the presence of other vehicles"],
    "Major traffic offense": ["Disrespecting the intersection", 
                              "Driver changed the lane illegally",
                              "Driver disrespected the red traffic light",
                              "Driver was in the opposite direction",
                              "Driving on the breakdown lane",
                              "Prohibited conversion"],
    "Driver distraction": ["Driver using cellphone",
                         "Driver was sleeping",
                         "Driver's lack of reaction",
                         "Driver's lack of attention to conveyance"],
    "Road defect":  ["Inadequate sinalization of the road",
                     "Curvy road", "No breakdown lanes", "Other flaws/problems in the road",
                     "Poor ilumination (of the road)",
                     "Road's defect",
                     "Roads with holes without cement",
                     "Sinking or ondulation in the pavement",
                     "Slippery track",
                     "Uneven breakdown lane",
                     "Unlevel track",
                     "Urban area without appropriate pedestrian walking"],
    "Road condition": ["Accumulation of water on the road", "Fog",
                       "Natural phenomena",
                       "Obstacle in the road",
                       "Oil accumulation on the road",
                       "Rain",
                       "Road had lots of sand/wreckage",
                       "Road works (in maintenance)",
                       "Static object on the drainage gate",
                       "Visibility restriction"],
    "Alcohol": ["Alcohol and/or drug ingestion by the pedestrian", "Alcohol consumption",
                "Alcohol ingestion by the driver"],
    "Drugs": ["Driver was using drugs"],
    "Driver behavior": ["External fight"],
    "Animals": ["Animals on the road"],
    "Veichle not human fault": ["Car's brake problem", 
                       "Car's suspension system with problems", 
                       "Deficiency of vehicle's sinalization/ilumination system",
                       "Electrical or mechanical flaws",
                       "Mechanical loss/defect of vehicle"],
    "Veichle human fault": ["Excessive load/cargo", "Excessive use of the car's tire"],
    "Driver health": ["Cardiac attack", "Driver had a cardiac attack"],
    "Safe distance": ["Disrespect of safe distance from the next car",
                      "Driver failed to keep distance from the vehicle in front"],
    "High speed": ["Incompatible velocity"],
    "Pedestrian involved": ["Pedestrian was crossing the road outside of the crosswalk",
                            "Pedestrian was walking in the road",
                            "Pedestrian's lack of attention",
                            "Unexpected pedestrian entry"]
    }
    
    # Since for Pandas it's more convenient to have the specific causes as key, we reverse the mapping of the dictionary
    reverse_mapping = {specific: general 
                    for general, specifics in cause_mapping.items() 
                    for specific in specifics}
    df_selection["general_cause_of_accident"] = df_selection["cause_of_accident"].map(reverse_mapping)
    
    
    for cause in cause_to_analyze:
        df_filtered = df_selection.query(f"general_cause_of_accident == '{cause}'").copy()
        gpsCoords = df_filtered[['latitude', 'longitude']].copy()
        print(f"OPTICS tuning, {param}: {query}, cause: {cause}")
        tuning_df = get_optics_tuning_df(cause, maxRadiusArr=maxRadius_to_test, minPtsArr=minPtsArr, xiArr = xi_to_test, data=df_filtered)
        algorithm_stats, labels = get_GeoOPTICS_metrics(1.0, 4, 0.1, gpsCoords)
        df_filtered = df_filtered.copy()
        df_filtered['labels'] = labels

        # Hotspot score
        data_to_plot = df_filtered[df_filtered['labels'] > -1].copy()
        # Adding a rank for victims_condition
        victims_condition_map = {
            "Without victims": 0,
            "With injured victims": 1,
            "With dead victims": 2
        }
        data_to_plot['victims_condition_rank'] = data_to_plot["victims_condition"].map(victims_condition_map)
        
        # Plot showing the clustering labels only
        print("Plot according to clustering labels")
        plot_map(data = data_to_plot, text = f'{param}: {query} - {cause} - cluster labels - OPTICS', color_col='labels')  # outliers are excluded

        # Computing the hotspot score
        group = data_to_plot.groupby("labels")
        accidents_per_label = group.size().reset_index(name="accidents_per_label") # it's a series
        sum_victims_condition_rank = group["victims_condition_rank"].sum().reset_index(name="sum_victims_condition_rank") 
        clusters_stats = pd.merge(accidents_per_label, sum_victims_condition_rank, on="labels")

        alpha = 0.2
        beta = 0.8
        clusters_stats['hotspot_score'] = alpha*clusters_stats['accidents_per_label'] + beta*clusters_stats['sum_victims_condition_rank']

        scaler = MinMaxScaler()
        clusters_stats["hotspot_score"] = scaler.fit_transform(clusters_stats[["hotspot_score"]])

        # Second plot
        print("Plot showing the hotspots:")
        plot_map2(data = data_to_plot, text = f'{param}: {query} - {cause} - hotspots - OPTICS', clusters_stats=clusters_stats)  # outliers are excluded

        return tuning_df

In [10]:
cause_to_analyze = ['High speed']
maxRadius_to_test = [1.0, 1.1, 1.2, 1.3, 1.4, 1.5, 1.7, 1.8, 1.85, 1.9, 2]
minPtsArr = [2,3,4]
xi_to_test = [0.07, 0.10, 0.11, 0.12, 0.13]

maxRadius = 1.0
minPts = 4
xi = 0.1
tuning_df = launch('State', 'RJ', cause_to_analyze, maxRadius_to_test, minPtsArr, xi_to_test, max_radiusUser=maxRadius, min_ptsUser=minPts, xiUser=xi)

OPTICS tuning, State: RJ, cause: High speed
Plot according to clustering labels


C:\Users\lorenzo\AppData\Local\Temp\ipykernel_9316\3660362575.py:20: DeprecationWarning: *scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/
  fig = px.scatter_mapbox(


Plot showing the hotspots:


C:\Users\lorenzo\AppData\Local\Temp\ipykernel_9316\4090563293.py:20: DeprecationWarning:

*scatter_mapbox* is deprecated! Use *scatter_map* instead. Learn more at: https://plotly.com/python/mapbox-to-maplibre/



In [12]:
tuning_df

,min_samples,max_radius,xi,count_core_points,count_outliers,corePoints_outliers_ratio,number_of_clusters,davies_bouldin_index,sil_coefficent,calinski_index
11,4,1.00,0.10,1452,1028,1.412451,161,0.291498,0.753107,165124.143473
12,4,1.00,0.11,1452,1028,1.412451,161,0.291498,0.753107,165124.143473
44,4,1.20,0.13,1455,1025,1.419512,165,0.287939,0.766292,155817.031143
43,4,1.20,0.12,1467,1013,1.448174,167,0.292240,0.759862,155188.775913
10,4,1.00,0.07,1476,1004,1.470120,163,0.299702,0.747257,161038.980902
...,...,...,...,...,...,...,...,...,...,...
136,2,1.90,0.10,2116,364,5.813187,541,0.340133,0.704630,147294.399385
120,2,1.85,0.07,2125,355,5.985915,544,0.343450,0.702087,149487.785938
151,2,2.00,0.10,2127,353,6.025496,544,0.340581,0.703748,139004.571797
135,2,1.90,0.07,2128,352,6.045455,544,0.345081,0.700891,146820.196301
